ПАКЕТ SURPRISE

− используйте данные MovieLens 1M
− можно использовать любые модели из пакета
− получите RMSE на тестовом сете 0.87 и ниже

In [1]:
import pandas as pd
import numpy as np

from surprise import KNNWithMeans, KNNBasic, NMF
from surprise import Dataset
from surprise import accuracy
from surprise import Reader

from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV


%matplotlib inline

In [2]:
movies = pd.read_csv('../movies.csv', )
ratings = pd.read_csv('../ratings.csv',)

In [3]:
movies_with_ratings = ratings.merge(movies, on='movieId', how='left')
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.movieId,
    'rating': movies_with_ratings.rating})

In [4]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [5]:
# Установим параметры алгоритма
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [6]:
trainset, testset = train_test_split(data, test_size=.25)

In [7]:

# define a cross-validation iterator
kf = KFold(n_splits=5)

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8859
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8794
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8755
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8870
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8807


In [8]:
algo.fit(trainset)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [9]:
predictions = algo.test(testset)

In [10]:
accuracy.rmse(predictions, verbose=True)

RMSE: 0.8807


0.8807279292812521

In [ ]:
#Используем grid search для подбора оптимальных значений:
ks = [30]
param_grid = {'k': ks, 
              'sim_options': {'name': ['msd', 'cosine', 'pearson_baseline'],
                              'user_based': [False, True]}}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
#Лучшее значение RMSE score
print(gs.best_score['rmse'])

#Комбинация параметров для лучшего значения RMSE score
print(gs.best_params['rmse'])